<a href="https://colab.research.google.com/github/kemusiro/ProgrammingGakuen-PythonClub/blob/main/202301/hash_collision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import hashlib
import io
import math
import random
from google.colab import drive
import IPython
from PIL import Image

# ハッシュ値を計算するためのクラス
class Hash:
    # 4ビット単位でハッシュ値の長さを指定する。
    def __init__(self, length):
        self.length = length

    # バイト配列に対してハッシュ値を計算する。
    def hash(self, values):
        return hashlib.md5(values).hexdigest()[-self.length:]

    # Imageオブジェクトの画像データ部分に対するハッシュ値を計算する。
    def hash_image(self, image):
        return self.hash(image.tobytes())

# 与えられた画像に対して一部が異なる画像データを生成する。
def generate(img, n):
    w, h = img.size
    images = []
    for _ in range(n):
        new_img = img.copy()
        x = random.randrange(0, w - 1)
        y = random.randrange(0, h - 1)
        new_img.putpixel((x, y), (0, 255, 0))
        images.append((new_img, (x, y)))
    return images

# Googleドライブをマウントする。
drive.mount("/content/drive")
path = "/content/drive/MyDrive/Colab Notebooks/"

# ハッシュ値の長さを20ビット(5*4ビット)とする。
length = 5
hasher = Hash(length)
print("maximum space = {}".format(2 ** (length * 4)))

# 2つの画像ファイルを開く
img1 = Image.open(path + "写真1.jpg")
print("image1: hash = {}".format(hasher.hash_image(img1)))

img2 = Image.open(path + "写真2.jpg")
print("image2: hash = {}".format(hasher.hash_image(img2)))

# 生成するバリエーション画像の数を決定する。
nimg = math.ceil(1.1774 * math.sqrt(2 ** (length * 4)))
print("generate {} images".format(nimg))

# 同じハッシュ値の画像ペアが見つかるまで最大10回繰り返す。
for i in range(10):
    print("iteration {}".format(i))
    # それぞれの画像ファイルに対するバリエーション画像を生成する。
    variation1 = generate(img1, nimg)
    variation2 = generate(img2, nimg)

    # バリエーション画像毎にハッシュ値を計算する。
    hv1 = {hasher.hash_image(e[0]): e for e in variation1}
    hv2 = {hasher.hash_image(e[0]): e for e in variation2}
    print("valid images: variation1 = {}, variation2 = {}".format(len(hv1), len(hv2)))

    # 同じハッシュ値となる画像を求める。
    intersection = set(hv1.keys()) & set(hv2.keys())
    if intersection:
        print("found hash value = {}".format(intersection))
        # 複数の候補がある場合は任意の一つを取り出す。
        sample = intersection.pop()
        # 取り出したハッシュ値に対する画像と、画像内で修正したピクセル位置を表示する。
        print("image 1: hash = {}, point = {}".format(sample, hv1[sample][1]))
        IPython.display.display(hv1[sample][0])
        hv1[sample][0].save(path + "modified-写真1.png")
        hv1[sample][0].save(path + "modified-写真1.jpg")
        print("image 2: hash = {}, point = {}".format(sample, hv2[sample][1]))
        IPython.display.display(hv2[sample][0])
        hv2[sample][0].save(path + "modified-写真2.png")
        hv2[sample][0].save(path + "modified-写真2.jpg")
        break
else:
    print("not found")